<a href="https://colab.research.google.com/github/sungjuGit/COSMOS_Ju/blob/main/AutoDiff_rev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.10.4" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia" # BenchmarkTools PyCall PyPlot"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  #if [ "$COLAB_GPU" = "1" ]; then
   #   JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  #fi
  #for PKG in `echo $JULIA_PACKAGES`; do
   # echo "Installing Julia package $PKG..."
   # julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  #done

  julia -e 'using Pkg; pkg"add IJulia; precompile";'

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key)"

Installing Julia 1.10.4 on the current Colab Runtime...
2024-07-04 22:51:12 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.10/julia-1.10.4-linux-x86_64.tar.gz [173704015/173704015] -> "/tmp/julia.tar.gz" [1]
┌ Warning: The Pkg REPL mode is intended for interactive use only, and should not be used from scripts. It is recommended to use the functional API instead.
└ @ Pkg.REPLMode /usr/local/share/julia/stdlib/v1.10/Pkg/src/REPLMode/REPLMode.jl:382
  Installing known registries into `~/.julia`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed JLLWrappers ───── v1.5.0
   Installed Parsers ───────── v2.8.1
   Installed ZMQ ───────────── v1.2.6
   Installed IJulia ────────── v1.25.0
   Installed JSON ──────────── v0.21.4
   Installed Conda ─────────── v1.10.2
   Installed PrecompileTools ─ v1.2.1
   Installed Preferences ───── v1.4.3
   Installed ZeroMQ_jll ────── v4.3.5+0
   Installed libsodium_jll ─── v1.0.20+0
   Inst

# Autodiff:  <br> Calculus  from another angle

## Babylonian sqrt

I would like to use a simple example, computation of sqrt(x), where for me how autodiff works came as both a mathematical surprise, and a computing wonder.  The example is  the Babylonian algorithm, known to man for millenia, to compute sqrt(x):  


 > Repeat $ t \leftarrow  (t+x/t) / 2 $ until $t$ converges to $\sqrt{x}$.

 Each iteration has one add and two divides. For illustration purposes, 10 iterations suffice.

In [1]:
function Babylonian(x; N = 10)
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end
    t
end

Babylonian (generic function with 1 method)

In [2]:
x=2; Babylonian(x),√x  # Type \sqrt+<tab> to get the symbol

(1.414213562373095, 1.4142135623730951)

## ...and now the derivative, almost by magic

In [4]:
struct D <: Number  # D is a function-derivative pair
    f::Tuple{Float64,Float64}
end

In [5]:
D(x,y) = D((x,y)) # teaching point, overload D to take in either a tuple, or two arguments

D

Sum Rule: (x+y)' = x' + y' <br>
Product Rule: (x*y)' = x'y + xy' <br>
Quotient Rule: (x/y)' = (yx'-xy') / y^2

In [6]:
import Base: +, -, *, /, >, convert, promote_rule
+(x::D, y::D) = D(x.f .+ y.f)
-(x::D, y:: D) = D(x.f .- y.f)
*(x::D, y::D) = D((x.f[1]*y.f[1], (x.f[2]*y.f[1] + x.f[1]*y.f[2])))
/(x::D, y::D) = D((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
>(x::D, y::D) = x.f[1] > y.f[1]
convert(::Type{D}, x::Real) = D((x,zero(x)))
promote_rule(::Type{D}, ::Type{<:Number}) = D

promote_rule (generic function with 133 methods)

In [7]:
x=49; Babylonian(D(x,1)), (√x,.5/√x)

(D((7.0, 0.07142857142857142)), (7.0, 0.07142857142857142))

## Dual Number Notation

Instead of D(a,b) we can write a + b ϵ, where ϵ satisfies ϵ^2=0.  (Some people like to recall imaginary numbers where an i is introduced with i^2=-1.)

Others like to think of how engineers just drop the O(ϵ^2) terms.

The four rules are

$ (a+b\epsilon) \pm (c+d\epsilon) = (a \pm c) +  (b \pm d)\epsilon$

$ (a+b\epsilon) * (c+d\epsilon) = (ac) + (bc+ad)\epsilon$

$ (a+b\epsilon) / (c+d\epsilon) = (a/c) + (bc-ad)/c^2 \epsilon $


In [8]:
Base.show(io::IO,x::D) = print(io,x.f[1]," + ",x.f[2]," ϵ")

In [9]:
D((1,0))

1.0 + 0.0 ϵ

In [10]:
D((2,1)) ^2

4.0 + 4.0 ϵ

In [11]:
ϵ = D((0,1))


0.0 + 1.0 ϵ

In [12]:
ϵ * ϵ

0.0 + 0.0 ϵ

In [13]:
ϵ^2


0.0 + 0.0 ϵ

In [14]:
1/(1+ϵ)  # Exact power series:  1-ϵ+ϵ²-ϵ³-...

1.0 + -1.0 ϵ

In [15]:
(1+ϵ)^5  ## Note this just works (we didn't train powers)!!

1.0 + 5.0 ϵ

In [16]:
(1+ϵ)^(-1)

1.0 + -1.0 ϵ